In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import networkx as nx
from itertools import combinations, product

## Задание 01 ( 5 баллов)
Рассмотрим сеть авиарейсов между различными аэропортами US в 2015. Добавьте не более 500 ребер в сеть, чтобы средний кратчайший путь стал наименьшим. Работайте с сильно связанной компонентой (AR_strong)

In [56]:
AR = nx.read_edgelist("airport.edgelist", data=True, create_using=nx.DiGraph())
print(AR.is_directed())
print(len(AR), len(AR.edges()))
print(nx.is_strongly_connected(AR))

True
1258 25354
False


In [3]:
AR_strong = AR.subgraph(max(nx.strongly_connected_components(AR), key=len))
print(AR_strong.number_of_nodes(), AR_strong.number_of_edges())

1190 25279


### Выведем среднее значение кратчайшего расстояния в системе: 

In [4]:
print(nx.average_shortest_path_length(AR_strong))

3.174661992635574


Отсортируем вершины по средней длине кратчайшего пути и соединим "слыбые" вершины с "сильными".

In [11]:
avg_path_lengths = {}

for node in list(AR_strong):
  avg_path_lengths[node] = np.mean(
      list(nx.single_source_shortest_path_length(AR_strong, node).values()))

In [84]:
import operator 
from collections import OrderedDict
from itertools import product

AR_strong_сompressed = AR_strong.copy()

n_hubs = 0;
n_edges_to_add = 500;
n_edges_to_connect_to_hubs = n_edges_to_add // 2;

avg_path_lengths_sorted = OrderedDict(sorted(avg_path_lengths.items(), 
                                             key=operator.itemgetter(1)))
keys = list(avg_path_lengths_sorted.keys())

for i in range(n_edges_to_connect_to_hubs):
  u = keys[-i]
  is_added = False
  is_reverse_added = False

  for v in keys:
    if v != u:
      if not AR_strong_сompressed.has_edge(u, v) and not is_added:
        AR_strong_сompressed.add_edge(u, v)
        is_added = True

      if not AR_strong_сompressed.has_edge(v, u) and not is_reverse_added:
        AR_strong_сompressed.add_edge(v, u)
        is_reverse_added = True

      if is_added and is_reverse_added:
        break

print(nx.average_shortest_path_length(AR_strong_сompressed))

2.728661893689351


In [85]:
len(AR_strong_сompressed.edges) - len(AR_strong.edges)

500

## Задание 02 Метрика качества разбиения на сообщества (4 балла)


### Матрица попарных ошибок:
$$
A=\begin{pmatrix}
 a_{00} &  a_{01}\\
a_{10}& a_{11}
\end{pmatrix}	
$$

Hассмотрим всевоозможные пары вершин:
* $a_{00}$ - количество пар, относящиеся к одному сообществу и в первом и во втором разбиениях;
* $a_{01}$ - количество пар, относящиеся к одному сообществу в первом разбиении и к разным во втором разбиении;
* $a_{10}$ - количество пар, относящиеся к одному сообществу во втором разбиении и к разным во первом разбиении;
* $a_{11}$ - количество пар, относящиеся к разным сообществам и в первом и во втором разбиениях;

Для сети каратэ и различных алгоритмов выделения сообществ (основанный на схожести, Лювен, спектральные методы, label propagation) определите матрицу попарных ошибок. 

In [ ]:
from itertools import combinations, product
 
def pairwise_same_label(partition):
  """Compute pairwise binary labels.
 
  Arguments:
  partition -- partition of network, dict {node: label}
 
  Returns:
  dict {(node1, node2): 1 if different labels,  otherwise 0}
  """
 
  if type(partition) != dict:
    raise ValueError("Partition must be a dict object")
 
  return {(u, v): int(not partition[u] == partition[v]) for u, v in
          combinations(partition.keys(), 2)}
 
def pairwise_error_matrix(part1, part2):
  """Compute pairwise error matrix.
 
  Arguments:
  part1 -- 1st partition of network, dict {node: label}
  part2 -- 2st partition of network, dict {node: label}
 
  Returns:
  Rairwise error matrix, np.array
  """
 
  pairs1, pairs2 = map(pairwise_same_label, (part1, part2))
  A = np.zeros((2, 2), dtype=int)
 
  for key, value1 in pairs1.items():
    try:
      value2 = pairs2[key]
    except KeyError:
      value2 = pairs2[(key[1], key[0])]
  
    A[value1][value2] += 1
   
  return A

In [ ]:
import operator
 
def similarity_matrix(G, nodelist=None):
  if (nodelist == None):
      nodelist = list(G.nodes)

  Smatrix = np.zeros([len(nodelist), len(nodelist)])

  for i in range(len(nodelist)):
      for j in range(i+1, len(nodelist)):
          u, v = nodelist[i], nodelist[j]
          k_u, k_v = len(G[u]), len(G[v])
          J_ij = len(set(G[u]) & set(G[v]))
          A_ij = int(G.has_edge(u, v))
          Smatrix[i, j] = (J_ij + A_ij) / (np.min(k_u, k_v) + 1 - A_ij)
          Smatrix[j, i] = Smatrix[i, j]

  return Smatrix
 
def ravatz(G, n, measure='mean'):
  nodes = list(G)
  N = len(nodes)
  if n == 1 or n > N - 1:
    raise ValueError('n must be between 2 and N - 1, where N is number of nodes')
 
  idx = {node: i for i, node in enumerate(nodes)}
 
  X = similarity_matrix(G)
 
  labels_to_coms = {i: frozenset([nodes[i]]) for i in range(N)}
  coms_to_labels = {value: key for key, value in labels_to_coms.items()}
  merges = []
 
  measures = {'mean': np.mean,
              'max': np.max,
              'min': np.min}
 
  def d(c1, c2):
    ds = np.array([])
    for u in c1:
      for v in c2:
        ds = np.append(ds, X[idx[u], idx[v]])
    return measures[measure](ds)
 
  while len(labels_to_coms) != 1:
    ds = {(c1, c2): d(c1, c2) for c1, c2 in combinations(labels_to_coms.values(), 2)}
    max_d = max(ds.items(), key=operator.itemgetter(1))[1]
    pos_coms_to_merge = [key for key, value in ds.items() if 
                           value == max_d]
 
    c1, c2 = pos_coms_to_merge[np.random.randint(len(pos_coms_to_merge))]
    label1, label2 = coms_to_labels[c1], coms_to_labels[c2]
    coms_to_labels[c1 | c2] = label1
    coms_to_labels.pop(c1)
    coms_to_labels.pop(c2)
    labels_to_coms.pop(label2)
    labels_to_coms[label1] = c1 | c2
    merges.append([set(value) for value in labels_to_coms.values()])
 
  return merges[-n]

In [ ]:
G = nx.karate_club_graph()
part = ravatz(G, 3)
part

[{0, 1, 2, 3, 4, 5, 6, 7, 10, 11, 12, 13, 16, 17, 19, 21},
 {8, 9, 14, 15, 18, 20, 22, 23, 26, 27, 28, 29, 30, 32, 33},
 {24, 25, 31}]

In [ ]:
def node_for_label(partition):
  res = {}
  for value, keys in enumerate(partition):
    for key in keys:
      res[key] = value
  return res

In [ ]:
node_for_label(part)

{0: 0,
 1: 0,
 2: 0,
 3: 0,
 4: 0,
 5: 0,
 6: 0,
 7: 0,
 8: 1,
 9: 1,
 10: 0,
 11: 0,
 12: 0,
 13: 0,
 14: 1,
 15: 1,
 16: 0,
 17: 0,
 18: 1,
 19: 0,
 20: 1,
 21: 0,
 22: 1,
 23: 1,
 24: 2,
 25: 2,
 26: 1,
 27: 1,
 28: 1,
 29: 1,
 30: 1,
 31: 2,
 32: 1,
 33: 1}

In [ ]:
from collections import Counter 
 
def highest_frequency_labels(v, labels, G):
    if not G[v]:
        return [labels[v]]
    
    label_freqs = Counter()
    for n in G[v]:
        label_freqs[labels[n]] += 1
            
    max_freq = max(label_freqs.values())
    return [label for label, freq in label_freqs.items() 
            if freq == max_freq]
 
def label_propagation(G):
    labels = {v: k for k, v in enumerate(G)}
    nodes = list(G)
    boo = True
    
    while boo:
        boo = False
        np.random.shuffle(nodes)
        
        for v in nodes:
            if not G[v]:
                continue
                
            labels_to_choose = highest_frequency_labels(v, labels, G)
            chosen_label = np.random.choice(labels_to_choose)
            labels[v] = chosen_label
        
        boo = any(labels[v] not in highest_frequency_labels(v, labels, G)
                 for v in nodes if G[v])
    return labels
 
 
def spectral(G):
  L = nx.laplacian_matrix(G)
  w, v = np.linalg.eig(L.toarray())
  idx = w.argsort()
  v2 = np.transpose(v)[idx][1]
  spectral = {i: int(v2[i - 1] >= 0) for i in G.nodes}  
 
  return [set((v for v in spectral if spectral[v] == label)) 
            for label in set(spectral.values())]

In [ ]:
import community as co
 
G = nx.karate_club_graph()
 
parts = {'Ravatz': node_for_label(ravatz(G, 2)),
         'Spectral': node_for_label(spectral(G)),
         'Louvain': co.best_partition(G),
         'Label propagation': label_propagation(G)}
 
print('Pairwise error matrices: ')
print('-' * 50)
 
for key1, key2 in combinations(parts.keys(), 2):
  print(key1, '---', key2, ': ')
  print(pairwise_error_matrix(parts[key1], parts[key2]))
  print('-' * 50)

Pairwise error matrices: 
--------------------------------------------------
Ravatz --- Spectral : 
[[148 125]
 [128 160]]
--------------------------------------------------
Ravatz --- Louvain : 
[[149 124]
 [ 11 277]]
--------------------------------------------------
Ravatz --- Label propagation : 
[[217  56]
 [ 13 275]]
--------------------------------------------------
Spectral --- Louvain : 
[[ 79 197]
 [ 81 204]]
--------------------------------------------------
Spectral --- Label propagation : 
[[120 156]
 [110 175]]
--------------------------------------------------
Louvain --- Label propagation : 
[[154   6]
 [ 76 325]]
--------------------------------------------------


## Задание 03 Рандомизация направленных сетей (6 баллов)

Будем работать с направленной сетью ассоциаций английского языка. 
* Определите долю двунаправленных связей в сети и транзитивность сети. 
* Постройте рандомизированную модель и вычислите как изменилась доля двунаправленных связей и транзитивность в модели. 
* Перепишите алгоритм рандомизации таким образом, чтобы число двунаправленных связей оставалось неизменным (но сами связи меняли свое положение!!!). Сравните транзитивность в этой модели. 

In [ ]:
import pandas as pd
net = pd.read_excel('assoc_eng2.xlsx', index_col=0)
net.head()

FileNotFoundError: ignored

In [ ]:
G = nx.DiGraph()
edges = zip(list(net['source']), list(net['target']))
G.add_edges_from(edges)
print(G.number_of_nodes(), G.number_of_edges())

5019 63629


In [ ]:
def bidirectional_edges(G):
  edges = set()
  nodes = list(G)

  for i in range(len(nodes)):
    for j in range(i + 1, len(nodes)):
      if (G.has_edge(nodes[i], nodes[j]) and G.has_edge(nodes[j], nodes[i])):
        edges |= {(nodes[i], nodes[j])}

  return edges

def count_bidirectional_edges(G):
  return len(bidirectional_edges(G))

print('Число двунаправленных связей: ', count_bidirectional_edges(G))
print('Транзитивность сети: ', nx.transitivity(G))

Число двунаправленных связей:  8383
Транзитивность сети:  0.1228734955845273


In [ ]:
import random

def random_rewiring(G, N_steps):
    G_rand = G.copy() 
       
    for n in range(N_steps):
        edges = list(G_rand.edges())
        
        switched = False

        while not switched:
            e1 = random.choice(edges)
            e2 = random.choice(edges)

            if (e1[0] == e2[1] or e1[1] == e2[0] or G_rand.has_edge(e1[0], e2[1]) 
              or G_rand.has_edge(e2[0], e1[1])):
                continue
            
            switched = True

            G_rand.add_edge(e1[0], e2[1])
            G_rand.add_edge(e2[0], e1[1])
            G_rand.remove_edge(*e1)
            G_rand.remove_edge(*e2)
            
    return G_rand

    
def bd_random_rewiring(G, N_steps):
    G_rand = G.copy() 
       
    for n in range(N_steps):
        edges = list(G_rand.edges())
        
        switched = False

        def has_bd_edge(u, v):
          return G_rand.has_edge(u, v) and G_rand.has_edge(v, u)

        while not switched:
            e1 = random.choice(edges)
            e2 = random.choice(edges)
            A, B, C, D = e1[0], e1[1], e2[0], e2[1]

            if (A in e2 or B in e2 or G_rand.has_edge(A, D) or 
                G_rand.has_edge(C, B) or G_rand.has_edge(D, A) or 
                G_rand.has_edge(B, C)):
              continue

            switched = True

            G_rand.add_edge(A, D)
            G_rand.add_edge(C, B)
            G_rand.remove_edge(A, B)
            G_rand.remove_edge(C, D)

            if G_rand.has_edge(D, C):
              G_rand.add_edge(D, A)
              G_rand.remove_edge(D, C)

            if G_rand.has_edge(B, A):
              G_rand.add_edge(B, C)
              G_rand.remove_edge(B, A)
            
    return G_rand

In [ ]:
G_rand1 = random_rewiring(G, 1000)

print('Число двунаправленных связей рандомизированной сети: ', 
      count_bidirectional_edges(G_rand1))
print('Транзитивность рандомизированной сети: ', nx.transitivity(G_rand1))

Число двунаправленных связей рандомизированной сети:  7860
Транзитивность рандомизированной сети:  0.1123812832164775


In [ ]:
G_rand2 = bd_random_rewiring(G, 1000)

print('Число двунаправленных связей рандомизированной сети: ', 
      count_bidirectional_edges(G_rand2))
print('Транзитивность рандомизированной сети: ', nx.transitivity(G_rand2))

Число двунаправленных связей рандомизированной сети:  8383
Транзитивность рандомизированной сети:  0.1082186633772163


In [ ]:
print('Число оставшихся на месте двунаправленных связей: ',
      len(bidirectional_edges(G_rand2) & bidirectional_edges(G)))

Число оставшихся на месте двунаправленных связей:  7842
